## Tarea #5
## Clase #6
### Curso: BigData, Ciencia de los Datos
    
### Alonso Nuñez Sanchez
### 27 Enero 2020

# Intro ML SUP en BD

Creación de la sesión Spark:

In [146]:
#import SparkSession
from pyspark.sql import SparkSession


In [147]:
# Crear el spark session object, llamarle "supervised_ml"
spark=SparkSession.builder.appName('supervised_ml').getOrCreate()


## Regression 

Carga de datos, archivo *Linear_regression_dataset.csv*:

In [148]:
# Carga de datos
df=spark.read.csv('Linear_regression_dataset.csv',inferSchema=True,header=True)


Se invocan las librerias correcpondientes a **LinearRegression**, asi como las de OneHotEncoder, StringIndexer, VectorAssembler:

In [149]:
# Importacion de libs y operaciones
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression



Se visualizan algunos datos:

In [150]:
print((df.count(), len(df.columns))) 

(1232, 6)


Se muestran los primeros 10 datos:

In [151]:
# primeros 10 datos
df.limit(10).toPandas().head(10)



,var_1,var_2,var_3,var_4,var_5,label
0,734,688,81,0.328,0.259,0.418
1,700,600,94,0.320,0.247,0.389
2,712,705,93,0.311,0.247,0.417
3,734,806,69,0.315,0.260,0.415
4,613,759,61,0.302,0.240,0.378
5,748,676,85,0.318,0.255,0.422
6,669,588,97,0.315,0.251,0.411
7,667,845,68,0.324,0.251,0.381
8,758,890,64,0.330,0.274,0.436
9,726,670,88,0.335,0.268,0.422


## Feature Engineering

Creamos un solo vector con todos los features i.e 'var_1', 'var_2', 'var_3', 'var_4', 'var_5', a este le llamaremos "features" y como salida colocamos a 'label':

In [152]:
# Vector Ensamblador

df_assembler = VectorAssembler(inputCols=['var_1', 'var_2', 'var_3', 'var_4', 'var_5'], outputCol="features")
df = df_assembler.transform(df)

In [153]:
# visulizacion de vector ensamblado compuesto por features y label
df.select(['features','label']).show(20,False)


+------------------------------+-----+
|features                      |label|
+------------------------------+-----+
|[734.0,688.0,81.0,0.328,0.259]|0.418|
|[700.0,600.0,94.0,0.32,0.247] |0.389|
|[712.0,705.0,93.0,0.311,0.247]|0.417|
|[734.0,806.0,69.0,0.315,0.26] |0.415|
|[613.0,759.0,61.0,0.302,0.24] |0.378|
|[748.0,676.0,85.0,0.318,0.255]|0.422|
|[669.0,588.0,97.0,0.315,0.251]|0.411|
|[667.0,845.0,68.0,0.324,0.251]|0.381|
|[758.0,890.0,64.0,0.33,0.274] |0.436|
|[726.0,670.0,88.0,0.335,0.268]|0.422|
|[583.0,794.0,55.0,0.302,0.236]|0.371|
|[676.0,746.0,72.0,0.317,0.265]|0.4  |
|[767.0,699.0,89.0,0.332,0.274]|0.433|
|[637.0,597.0,86.0,0.317,0.252]|0.374|
|[609.0,724.0,69.0,0.308,0.244]|0.382|
|[776.0,733.0,83.0,0.325,0.259]|0.437|
|[701.0,832.0,66.0,0.325,0.26] |0.39 |
|[650.0,709.0,74.0,0.316,0.249]|0.386|
|[804.0,668.0,95.0,0.337,0.265]|0.453|
|[713.0,614.0,94.0,0.31,0.238] |0.404|
+------------------------------+-----+
only showing top 20 rows



Partimos a continuación el set de datos en 75% training y 25% testing:

In [154]:
# Particion del data set
model_df=df.select(['features','label'])
train,test = model_df.randomSplit([0.75,0.25])

print(f"Size of train Dataset : {train.count()}" )
print(f"Size of test Dataset : {test.count()}" )

Size of train Dataset : 896
Size of test Dataset : 336


Creamos el Regresor Lineal: 

In [155]:
lr = LinearRegression()

Entrenamos el modelo de regresión lineal:

In [156]:
# Fit the model, le llamamos lr_model
lr_model=lr.fit(train)



Creamos el dataframe de prediciones (*predictions_df*) a partir del modelo de entrenamiento y el conjunto de datos test: 

In [157]:
predictions_df = lr_model.transform(test)

Visualizamos el contenido de *predictions_df*:

In [158]:
# visulizacion de predictions_df
predictions_df.groupBy('label').count().show()


+-----+-----+
|label|count|
+-----+-----+
|0.445|    1|
|0.319|    1|
|0.413|    2|
|0.343|    1|
|0.394|    3|
|0.396|    2|
|0.373|    3|
|0.369|    2|
|0.317|    1|
|0.423|    2|
|0.329|    2|
|0.318|    1|
|0.399|    2|
|0.352|    4|
|0.416|    2|
|0.401|    5|
|0.415|    4|
|0.327|    1|
|0.392|    1|
|0.419|    3|
+-----+-----+
only showing top 20 rows



Ahora, evaluamos el modelo de Regresión Lineal, con los datos de TEST:

In [159]:
# evaluacion del modelo, le llamaremos model_predictions
model_predictions = lr_model.evaluate(test)



Imprimimos el valor de R2:

In [160]:
# valor de R2
trainingSummary = lr_model.summary

print("r2: %f" % trainingSummary.r2)


r2: 0.867061


Imprimimos el valor del meanSquaredError:

In [161]:
# valor del meanSquaredError
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)



RMSE: 0.011987


In [162]:
# otras métricas:

# Print the coefficients and intercept for linear regression

print("Coefficients: %s" % str(lr_model.coefficients))
print("Intercept: %s" % str(lr_model.intercept))


Coefficients: [0.0003336210820382659,6.158717053157263e-05,0.00023087339062401296,-0.6568906388796751,0.4980785935860538]
Intercept: 0.18137334024137847


## Regresión con Árboles de Decisión

Importamos la librería *DecisionTreeRegressor*: 

In [163]:
# import lib
from pyspark.ml import Pipeline
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.feature import VectorIndexer




Creamos el Regresor DT, le llamaremos *dec_tree*:

In [164]:
# dec_tree
dec_tree = DecisionTreeRegressor(featuresCol="features")



Entrenamos el modelo:

In [165]:
# Train model, le llamaremos dec_tree_model
dec_tree_model = dec_tree.fit(train)


Cuánto es la profundidad máxima por defecto, de este algoritmo?

### Respuesta:
el parámetro max_depth por defecto está configurado en 5

Desplegamos las *featureImportances*:

In [226]:
print(dec_tree_model.featureImportances)

(5,[0,1,2,3,4],[0.9642661892532784,0.017493134111902627,0.0019448235832598278,0.0024390255051261146,0.013856827546432877])


Evaluamos el modelo con los datos de entrenamiento:

In [167]:
# Make predictions, le llamaremos model_predictions 
model_predictions = dec_tree_model.transform(test)



In [168]:
# visualizamos

model_predictions.show()

+--------------------+-----+-------------------+
|            features|label|         prediction|
+--------------------+-----+-------------------+
|[468.0,746.0,52.0...|0.329| 0.3273333333333334|
|[486.0,610.0,61.0...|0.332|0.31800000000000006|
|[510.0,588.0,72.0...|0.317|0.31800000000000006|
|[511.0,576.0,76.0...|0.329|0.31800000000000006|
|[528.0,652.0,71.0...|0.319| 0.3273333333333334|
|[543.0,615.0,76.0...|0.333|0.31800000000000006|
|[543.0,747.0,60.0...|0.342|              0.355|
|[544.0,551.0,82.0...|0.344|0.33955555555555555|
|[550.0,631.0,76.0...|0.318|0.33955555555555555|
|[558.0,688.0,67.0...| 0.35|0.34913636363636363|
|[567.0,587.0,84.0...|0.349|0.34913636363636363|
|[568.0,708.0,57.0...|0.347|0.34913636363636363|
|[569.0,620.0,77.0...|0.349|0.34913636363636363|
|[569.0,711.0,65.0...| 0.34|0.34913636363636363|
|[573.0,656.0,75.0...|0.345|0.34913636363636363|
|[574.0,586.0,81.0...| 0.36|0.34913636363636363|
|[576.0,759.0,57.0...| 0.35|             0.3788|
|[578.0,633.0,76.0..

Importamos el **RegressionEvaluator**

In [169]:
# import Evaluator
from pyspark.ml.evaluation import RegressionEvaluator


Usando *RegressionEvaluator* calculamos e imprimimos el valor de las metricas R2 y RMSE:

In [170]:
# R2 value of the model on test data 
dt_evaluator = RegressionEvaluator(metricName='r2')
dt_r2 = dt_evaluator.evaluate(model_predictions)
print(f'The r-square value of DecisionTreeRegressor is {dt_r2}')

# RMSE value of the model on test data 
dt_evaluator_rmse = RegressionEvaluator(metricName='rmse')
rmse = dt_evaluator_rmse.evaluate(model_predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)




The r-square value of DecisionTreeRegressor is 0.8410597140380105
Root Mean Squared Error (RMSE) on test data = 0.0136459


## RandomForestRegressor

Importamos a *RandomForestRegressor*

In [171]:
# import lib
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator



Creamos el Regresor RF:

In [172]:
# Regresor 

rf = RandomForestRegressor(featuresCol="features")


Entrenamos el modelo:

In [173]:
# Train model, le llamaremos rf_model
rf_model = rf.fit(train)


Desplegamos las *featureImportances*:

In [223]:
# importances 
print(rf_model.featureImportances)



(5,[0,1,2,3,4],[0.49362086906566977,0.056194941602342986,0.03261255186079704,0.1916095114420692,0.225962126029121])


Desplegamos el numero de arboles (Num of Trees)

In [228]:
# Numero de Trees
print(rf_model.numTrees)


RandomForestRegressor_14d21afb5367__numTrees


Evaluamos el modelo con los datos de entrenamiento, le llamaremos model_predictions:


In [176]:
# model_predictions

model_predictions = rf_model.transform(test)


Desplegamos los valores del *model_predictions*

In [177]:
model_predictions.show()

+--------------------+-----+-------------------+
|            features|label|         prediction|
+--------------------+-----+-------------------+
|[468.0,746.0,52.0...|0.329| 0.3320035441070367|
|[486.0,610.0,61.0...|0.332| 0.3215623970158238|
|[510.0,588.0,72.0...|0.317| 0.3256053027650454|
|[511.0,576.0,76.0...|0.329|0.32208239701582386|
|[528.0,652.0,71.0...|0.319| 0.3327546912931839|
|[543.0,615.0,76.0...|0.333|0.32208239701582386|
|[543.0,747.0,60.0...|0.342|0.34786874646684013|
|[544.0,551.0,82.0...|0.344| 0.3455973167217622|
|[550.0,631.0,76.0...|0.318|0.34379262760369067|
|[558.0,688.0,67.0...| 0.35|  0.347142071999506|
|[567.0,587.0,84.0...|0.349|0.35544802373132517|
|[568.0,708.0,57.0...|0.347|0.36023419965808945|
|[569.0,620.0,77.0...|0.349| 0.3547970467789357|
|[569.0,711.0,65.0...| 0.34|0.35137002135648226|
|[573.0,656.0,75.0...|0.345|0.35107295876010214|
|[574.0,586.0,81.0...| 0.36|  0.351410791588468|
|[576.0,759.0,57.0...| 0.35|0.36932882837059555|
|[578.0,633.0,76.0..

Usando *RegressionEvaluator* calculamos e imprimimos el valor de las metricas R2 y RMSE:

In [178]:
# R2 value of the model on test data 
rf_evaluator = RegressionEvaluator(metricName='r2')
rf_r2 = rf_evaluator.evaluate(model_predictions)
print(f'The r-square value of DecisionTreeRegressor is {rf_r2}')

# RMSE value of the model on test data 
rf_evaluator_rmse = RegressionEvaluator(metricName='rmse')
rf_rmse = rf_evaluator_rmse.evaluate(model_predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rf_rmse)





The r-square value of DecisionTreeRegressor is 0.845264599879449
Root Mean Squared Error (RMSE) on test data = 0.0134641


## Gradient-Boosted Tree Regressor

Importamos a GBTRegressor


In [179]:
# import
from pyspark.ml import Pipeline
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator



Creamos el Regresor GBTR, le llamaremos gbt:


In [180]:
# regresor
gbtr = GBTRegressor(featuresCol="features", maxIter=10)



Entrenamos el modelo:

In [181]:
# Train model, le llamaremos gbt_model

gbt_model = gbtr.fit(train)




Desplegamos las featureImportances:

In [225]:
#Importances
print(gbt_model.featureImportances)



(5,[0,1,2,3,4],[0.3121458420400407,0.15735378280936319,0.12342456328662985,0.2061824671760019,0.2008933446879644])


Evaluamos el modelo con los datos de entrenamiento, le llamaremos model_predictions:

In [183]:
# Model

model_predictions = gbt_model.transform(test)



Desplegamos los valores del *model_predictions*

In [184]:
# show 
model_predictions.show()



+--------------------+-----+-------------------+
|            features|label|         prediction|
+--------------------+-----+-------------------+
|[468.0,746.0,52.0...|0.329|0.32675530727704505|
|[486.0,610.0,61.0...|0.332|0.31745622324052286|
|[510.0,588.0,72.0...|0.317|0.31838257488043564|
|[511.0,576.0,76.0...|0.329|0.31905434876653294|
|[528.0,652.0,71.0...|0.319|0.32885231982405133|
|[543.0,615.0,76.0...|0.333| 0.3195021143949359|
|[543.0,747.0,60.0...|0.342|0.35427049155523865|
|[544.0,551.0,82.0...|0.344| 0.3394122839935331|
|[550.0,631.0,76.0...|0.318| 0.3410576699504914|
|[558.0,688.0,67.0...| 0.35| 0.3495156071390776|
|[567.0,587.0,84.0...|0.349| 0.3506384780312995|
|[568.0,708.0,57.0...|0.347| 0.3492884808051989|
|[569.0,620.0,77.0...|0.349|0.35159528648389565|
|[569.0,711.0,65.0...| 0.34| 0.3484068551916023|
|[573.0,656.0,75.0...|0.345| 0.3450754884534634|
|[574.0,586.0,81.0...| 0.36| 0.3506384780312995|
|[576.0,759.0,57.0...| 0.35| 0.3789521171688353|
|[578.0,633.0,76.0..

Usando RegressionEvaluator calculamos e imprimimos el valor de las metricas R2 y RMSE:

In [185]:
 #Select (prediction, true label) and compute test error
    
model_predictions.select("prediction", "label", "features").show(5)

# R2 value of the model on test data 
gbt_evaluator = RegressionEvaluator(metricName='r2')
gbt_r2 = gbt_evaluator.evaluate(model_predictions)
print(f'The r-square value of DecisionTreeRegressor is {gbt_r2}')

# RMSE value of the model on test data 
gbt_evaluator_rmse = RegressionEvaluator(metricName='rmse')
gbt_rmse = gbt_evaluator_rmse.evaluate(model_predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % gbt_rmse)






+-------------------+-----+--------------------+
|         prediction|label|            features|
+-------------------+-----+--------------------+
|0.32675530727704505|0.329|[468.0,746.0,52.0...|
|0.31745622324052286|0.332|[486.0,610.0,61.0...|
|0.31838257488043564|0.317|[510.0,588.0,72.0...|
|0.31905434876653294|0.329|[511.0,576.0,76.0...|
|0.32885231982405133|0.319|[528.0,652.0,71.0...|
+-------------------+-----+--------------------+
only showing top 5 rows

The r-square value of DecisionTreeRegressor is 0.8455633775802
Root Mean Squared Error (RMSE) on test data = 0.0134511


 ## Exploracion de datos...

Usaremos el dataset https://archive.ics.uci.edu/ml/datasets/Bank+Marketing 

Indique a grandes razgos de que se trata este dataset:


### Respuesta:
Aparentemente son datos de clientes de un banco para clasificarlos como elegibles o no para una campaña de marketing

Carga de datos, archivo bank_data.csv:


In [186]:
# Load csv Dataset 
df=spark.read.csv('bank_data.csv',inferSchema=True,header=True)

Determine la cantidad de datos en el dataset:

In [187]:
#number of records

df.count()



41188

A que dato corresponde cada columna?

In [188]:
# columns values

df.columns

['age',
 'job',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'month',
 'day_of_week',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'poutcome',
 'emp.var.rate',
 'cons.price.idx',
 'cons.conf.idx',
 'euribor3m',
 'nr.employed',
 'target_class']

Imprima el Schema:

In [189]:
#dataype of input data - Schema

df.printSchema()



root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- month: string (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- emp.var.rate: double (nullable = true)
 |-- cons.price.idx: double (nullable = true)
 |-- cons.conf.idx: double (nullable = true)
 |-- euribor3m: double (nullable = true)
 |-- nr.employed: double (nullable = true)
 |-- target_class: string (nullable = true)



En cuanto a la salida, como es la distrubución de clases?

In [190]:
# YES/NO Class Distribution

df.groupBy('target_class').count().show()



+------------+-----+
|target_class|count|
+------------+-----+
|          no|36548|
|         yes| 4640|
+------------+-----+



Una tarea típica, resulta de convertir los valores binarios en 1 y 0, usando como referencia "label", convierta los no/yes en 0/1:

In [191]:
from pyspark.sql import functions as F
from pyspark.sql import *

In [192]:
# Ingrese acá la instrucción: 


df = df.withColumn("target_class", F.when(F.col("target_class")=='no', 0).otherwise(1))



In [193]:
# New 1/0 Class Distribution

df.groupBy('target_class').count().show()



+------------+-----+
|target_class|count|
+------------+-----+
|           1| 4640|
|           0|36548|
+------------+-----+



A continuación se presenta un ejercicio de Deep Learning para su revisión...

# Deep Learning 

Importamos las librerias necesarias:

In [194]:
import os
import numpy as np
import pandas as pd
from pyspark.sql.types import *
from pyspark.ml import Pipeline
from pyspark.sql import functions as f
from pyspark.sql.functions import udf, StringType
from pyspark.sql import SparkSession, functions as F
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.feature import OneHotEncoder, VectorAssembler, StringIndexer

Inicializamos la sesion SPARK:

In [195]:
spark = SparkSession.builder.appName('deep_learning').getOrCreate()

Leemos el dataset:

In [196]:
data = spark.read.csv('dl_data.csv', header=True, inferSchema=True)

In [197]:
data.printSchema()

root
 |-- Visit_Number_Bucket: string (nullable = true)
 |-- Page_Views_Normalized: double (nullable = true)
 |-- Orders_Normalized: integer (nullable = true)
 |-- Internal_Search_Successful_Normalized: double (nullable = true)
 |-- Internal_Search_Null_Normalized: double (nullable = true)
 |-- Email_Signup_Normalized: double (nullable = true)
 |-- Total_Seconds_Spent_Normalized: double (nullable = true)
 |-- Store_Locator_Search_Normalized: double (nullable = true)
 |-- Mapped_Last_Touch_Channel: string (nullable = true)
 |-- Mapped_Mobile_Device_Type: string (nullable = true)
 |-- Mapped_Browser_Type: string (nullable = true)
 |-- Mapped_Entry_Pages: string (nullable = true)
 |-- Mapped_Site_Section: string (nullable = true)
 |-- Mapped_Promo_Code: string (nullable = true)
 |-- Maped_Product_Name: string (nullable = true)
 |-- Mapped_Search_Term: string (nullable = true)
 |-- Mapped_Product_Collection: string (nullable = true)



Renombramos la columna TARGET:

In [198]:
data = data.withColumnRenamed('Orders_Normalized', 'label')

In [199]:
data.printSchema()

root
 |-- Visit_Number_Bucket: string (nullable = true)
 |-- Page_Views_Normalized: double (nullable = true)
 |-- label: integer (nullable = true)
 |-- Internal_Search_Successful_Normalized: double (nullable = true)
 |-- Internal_Search_Null_Normalized: double (nullable = true)
 |-- Email_Signup_Normalized: double (nullable = true)
 |-- Total_Seconds_Spent_Normalized: double (nullable = true)
 |-- Store_Locator_Search_Normalized: double (nullable = true)
 |-- Mapped_Last_Touch_Channel: string (nullable = true)
 |-- Mapped_Mobile_Device_Type: string (nullable = true)
 |-- Mapped_Browser_Type: string (nullable = true)
 |-- Mapped_Entry_Pages: string (nullable = true)
 |-- Mapped_Site_Section: string (nullable = true)
 |-- Mapped_Promo_Code: string (nullable = true)
 |-- Maped_Product_Name: string (nullable = true)
 |-- Mapped_Search_Term: string (nullable = true)
 |-- Mapped_Product_Collection: string (nullable = true)



Partimos lo datos en Train, Validation y Test:

In [200]:
train, validation, test  = data.randomSplit([0.7, 0.2, 0.1], 1234)

Construimos el Pipeline

In [201]:
categorical_columns = [item[0] for item in data.dtypes if item[1].startswith('string')]
numeric_columns = [item[0] for item in data.dtypes if item[1].startswith('double')]

indexers = [StringIndexer(inputCol=column, outputCol='{0}_index'.format(column)) for column in categorical_columns]

featuresCreator = VectorAssembler(inputCols=[indexer.getOutputCol() for indexer in indexers] + numeric_columns, outputCol="features")

layers = [len(featuresCreator.getInputCols()), 4, 2, 2]

classifier = MultilayerPerceptronClassifier(labelCol='label', featuresCol='features', maxIter=1000, layers=layers, blockSize=1024, seed=1234)

pipeline = Pipeline(stages=indexers + [featuresCreator, classifier])

Entrenamos...

In [202]:
model = pipeline.fit(train)

Validamos y Evaluamos

In [203]:
train_output_df = model.transform(train)
validation_output_df = model.transform(validation)
test_output_df = model.transform(test)

Llevamos a cabo, algunas predicciones:

In [204]:
train_predictionAndLabels = train_output_df.select("prediction", "label")
validation_predictionAndLabels = validation_output_df.select("prediction", "label")
test_predictionAndLabels = test_output_df.select("prediction", "label")

metrics = ['weightedPrecision', 'weightedRecall', 'accuracy']

for metric in metrics:
    evaluator = MulticlassClassificationEvaluator(metricName=metric)
    print('Train ' + metric + ' = ' + str(evaluator.evaluate(train_predictionAndLabels)))
    print('Validation ' + metric + ' = ' + str(evaluator.evaluate(validation_predictionAndLabels)))
    print('Test ' + metric + ' = ' + str(evaluator.evaluate(test_predictionAndLabels)))

Train weightedPrecision = 0.9737665435248012
Validation weightedPrecision = 0.9746155843208265
Test weightedPrecision = 0.9782946313349752
Train weightedRecall = 0.9729656032098477
Validation weightedRecall = 0.9738482384823848
Test weightedRecall = 0.9776574136763709
Train accuracy = 0.9729656032098477
Validation accuracy = 0.9738482384823848
Test accuracy = 0.977657413676371


Puede mejorar el test accuracy del modelo variando alguno de los hyperparametros?

 ### Respuesta:
 Sí puede mejorarse, algunos resultados cambiando maxIter y blockSize:
 
 maxIter=100, layers=layers, blockSize=128, seed=1234
 Accuracy = 0.9729
 
 maxIter=200, layers=layers, blockSize=256, seed=1234
 Accuracy = 0.9737
 
 maxIter=400, layers=layers, blockSize=512, seed=1234
 Accuracy = 0.9779  <--- este fue el mayor valor obtenido con las pruebas realizadas
 
 maxIter=1000, layers=layers, blockSize=1024, seed=1234
 Accuracy = 0.9776
 